In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Single family model
# made up of home and events
# start with a weekly calendar
# assign probability distributions

In [42]:
class Event():
    """ Base class """
    def __init__(self, people, duration, peer_pressure, volume):
        """ init """
        # variables between 0 and 1
        self.people = people
        self.case_density = 0.05 # 1 in 20 infected and transmissable

        # Variables for event space, m^3
        self.volume = volume
        self.ventilation = 0.01 # 1 outside, 0 inside, small space no ventilation
        self.loud = 0.5 # 1 = loud choir. 0.5 = loud cafe. 0.1 = library?
        self.mask = 0.05  # 5% mask wearing
        self.test = 0.01  # 1% test prior to attendance
        self.duration = duration # how long in this place
        self.peer_pressure = 0.1 # How likely will I wear a mask given most are not?
        self.starting_virus_density = 0 # 0 means no viable virus in room to start.

        # Variables for me
        self.immunity = 0.1 # 1 in 10 chance of no infection even with virus particles
        self.mask_dilution = 0.25 # guess. Mask only allows 25% particles to flow
        
        # Calculations
        self.people_infected = np.round(self.people * self.case_density)
        self._people_infected()
        self.virus_met = 0
        self._virus_met()

    def _people_infected(self):
        """ probability of people being transmissiable """
        mean_infected = self.case_density * self.people
        scenarios = np.random.binomial(self.people, self.case_density, 1000)
        self.people_infected = np.random.choice(scenarios, size=None)

    def _virus_met(self):
        """ If I assume infected people are there all the time, will the
        virus reach me in enough quantity? This is extrmely finger in air
        calibration.
        
        Particles diffuse over time. Ventilation acts in the opposite direction
        and may take away more than is added.
        
        Masks then dilute any that are met.
        
        I'm going to use this study to start:
            https://jamanetwork.com/journals/jamanetworkopen/fullarticle/2768712
            
        I am also going to assume a threshold of 300 particles for Omicron to trigger
        infection - with some binomial randomisation
        """
        pass

    def _breath(self):
        """ particle load per breath """
        pass

    def _brownian_motion(self):
        """ Model diffusion using brownian motion. Assume individuals have a fixed production
        of virus per unit time. This spreads through a volume over time increasing the density.
        But it is diluted by ventilation. """
        pass

In [43]:
ww_cafe = Event(people=25, duration=120, peer_pressure=0.1, volume=6*20)
ww_cafe.people_infected

3